<a href="https://colab.research.google.com/github/Hami-611/21Days_AI-ML_Challenge/blob/main/Day_15_Talk_to_Your_Data_Building_a_Natural_Language_to_SQL_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
## Text2SQL via Prompt Engineering on employee dataset


We have created 3 tables: `employee`, `department`, and `project`.
- The `employee` table has foreign keys for `department_name` and `manager_name`.
- The `project` table also has a foreign key for `department_name`.

## Retrieve Data

In [37]:
! curl "https://api.mockaroo.com/api/f8475b30?count=1000&key=77450f90" > "Department.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 24915    0 24915    0     0  21164      0 --:--:--  0:00:01 --:--:-- 21168


In [38]:
! curl "https://api.mockaroo.com/api/7003c510?count=1000&key=77450f90" > "employees.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80616    0 80616    0     0  22701      0 --:--:--  0:00:03 --:--:-- 22715


In [39]:
! curl "https://api.mockaroo.com/api/ee13d5b0?count=1000&key=77450f90" > "projects.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 44853    0 44853    0     0  24754      0 --:--:--  0:00:01 --:--:-- 24753


## Import required libraries

In [40]:
import sqlite3
import pandas as pd
import os

## Define SQL Schemas

In [41]:
employee_schema = """
create table if not exists employee (
	employee_id INT,
	first_name VARCHAR(50),
	last_name VARCHAR(50),
	email VARCHAR(50),
	hire_date DATE,
	salary INT,
	manager_name VARCHAR(50),
	department_name VARCHAR(50)
);
"""

department_schema = """
create table if not exists department (
	department_name VARCHAR(9),
	department_head VARCHAR(50),
	department_id INT
);
"""

project_schema = """
create table if not exists project (
	project_id INT,
	start_date DATE,
	end_date DATE,
	manager_name VARCHAR(50),
	department_name VARCHAR(50)
);
"""

In [42]:
db_name = 'employees.db'
if os.path.exists(db_name):
    os.remove(db_name)
    print(f"Removed existing database '{db_name}'.")

In [43]:
import sqlite3
import pandas as pd
import os

COLUMN_DATA_TYPES = {
    'employee': ['INT', 'VARCHAR(50)', 'VARCHAR(50)', 'VARCHAR(50)', 'DATE', 'INT', 'VARCHAR(50)', 'VARCHAR(50)'],
    'department': ['VARCHAR(9)', 'VARCHAR(50)', 'INT'],
    'project': ['INT', 'DATE', 'DATE', 'VARCHAR(50)', 'VARCHAR(50)']
}

TABLES = {
    'employee': 'employee',
    'department': 'department',
    'project': 'project'
}

## Database setup

In [44]:
db_name = 'employees.db'
conn = None

In [45]:
try:
    # Establish a connection to the SQLite database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    print(f"Database '{db_name}' created and connected successfully. ✅")

    # Create tables - using IF NOT EXISTS as added before
    cursor.execute(employee_schema)
    cursor.execute(department_schema)
    cursor.execute(project_schema)

    print("Tables 'employee', 'department', and 'projects' created successfully.")

    # --- Load data from CSV files into the tables using pandas ---
    csv_to_table_map = {
        '/content/employees.csv': 'employee',
        '/content/Department.csv': 'department',
        '/content/projects.csv': 'project'
    }

    # Define a dictionary to map table names to their actual column names from schemas
    # This is more reliable than parsing the schema strings repeatedly
    table_column_names = {
        'employee': ['employee_id', 'first_name', 'last_name', 'email', 'hire_date', 'salary', 'manager_name', 'department_name'],
        'department': ['department_name', 'department_head', 'department_id'],
        'project': ['project_id', 'start_date', 'end_date', 'manager_name', 'department_name']
    }


    for csv_file, table_name in csv_to_table_map.items():
        if os.path.exists(csv_file):
            print(f"\nProcessing '{csv_file}' for table '{table_name}'...")

            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(csv_file)

            # Get the expected column names for the current table from the new dictionary
            expected_col_names = table_column_names[table_name]

            # 2. Handle missing/extra columns
            # Drop columns from DataFrame that are not in the schema
            # Use actual column names for intersection
            df = df[df.columns.intersection(expected_col_names)]

            # Add any missing columns and fill with None (which becomes NULL in SQL)
            for col in expected_col_names:
                if col not in df.columns:
                    df[col] = None

            # 3. Reorder columns to match the defined schema exactly
            df = df[expected_col_names]

            # 4. Enforce data types
            # Use COLUMN_DATA_TYPES for type information, but iterate over expected_col_names
            expected_col_types = COLUMN_DATA_TYPES[table_name]

            for i, col in enumerate(expected_col_names):
                # Find the corresponding type string using the index
                dtype_str = expected_col_types[i]
                dtype = dtype_str.split(' ')[0] # Extract the base data type (e.g., 'INT', 'VARCHAR')

                if 'DATE' in dtype.upper(): # Check for DATE case-insensitively
                    # Use pd.to_datetime for date/time columns, coercing errors to NaT (Not a Time)
                    df[col] = pd.to_datetime(df[col], errors='coerce')
                elif 'INT' in dtype.upper(): # Check for INT case-insensitively
                     # Use astype for other columns, handling potential conversion errors
                    try:
                        # Attempt conversion to integer, coercing errors to NaN
                        df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64') # Use nullable Int64 to handle NaN
                    except Exception as e:
                         print(f"  - Warning: Could not convert column '{col}' to {dtype}. Error: {e}. Leaving as is.")
                # Add more elif conditions for other data types if needed
                # else:
                    # Default case if no specific type handling is needed

            # Use the to_sql method to insert the cleaned DataFrame
            # table_name should be the actual SQL table name
            df.to_sql(TABLES[table_name], conn, if_exists='append', index=False)
            print(f"  -> Data from '{csv_file}' loaded into '{TABLES[table_name]}' table successfully.")
        else:
            print(f"Warning: '{csv_file}' not found. Skipping data load for '{table_name}'.")

    # Commit the changes to the database
    conn.commit()
    print("\nData committed to the database successfully. 🎉")

except sqlite3.Error as e:
    print(f"Database error: {e}")
except pd.errors.EmptyDataError as e:
    print(f"Pandas error: {e}. One of the CSV files might be empty.")
except KeyError as e:
    print(f"Schema or table name mapping error: {e}. Check COLUMN_DATA_TYPES or TABLES dictionary.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    # Close the connection if it was established
    if conn:
        conn.close()
        print("Database connection closed.")

Database 'employees.db' created and connected successfully. ✅
Tables 'employee', 'department', and 'projects' created successfully.

Processing '/content/employees.csv' for table 'employee'...
  -> Data from '/content/employees.csv' loaded into 'employee' table successfully.

Processing '/content/Department.csv' for table 'department'...
  -> Data from '/content/Department.csv' loaded into 'department' table successfully.

Processing '/content/projects.csv' for table 'project'...
  -> Data from '/content/projects.csv' loaded into 'project' table successfully.

Data committed to the database successfully. 🎉
Database connection closed.


## Setup API Key

In [46]:
from google import genai
from google.colab import userdata

In [47]:
genai_client = genai.Client(api_key=userdata.get('Test2sql_Key'))

## Prompt

In [48]:
prompt = """
### **ROLE**

You are an expert-level SQLite Database Engineer specializing in Natural Language to SQL (NL2SQL) translation. Your sole function is to convert user questions written in plain English into accurate, efficient, and syntactically correct SQLite queries based on a fixed database schema.

-----

### **CONTEXT**

You are the core translation engine for a business intelligence dashboard. This tool allows non-technical employees to query the company's employee database using natural language. The database dialect is always **SQLite**. Your responses will be executed directly on the database.

The database consists of the following three tables:

**`employee` table:**

```sql
CREATE TABLE employee (
	employee_id INT,
	first_name VARCHAR(50),
	last_name VARCHAR(50),
	email VARCHAR(50),
	hire_date DATE,
	salary INT,
	manager_name VARCHAR(50),
	department_name VARCHAR(50)
);
```

**`department` table:**

```sql
CREATE TABLE department (
  department_name VARCHAR(9),
  department_head VARCHAR(50),
  department_id INT
);
```

**`project` table:**

```sql
CREATE TABLE project (
	project_id INT,
	start_date DATE,
	end_date DATE,
	manager_name VARCHAR(50),
	department_name VARCHAR(50)
);
```
**Relationships:**
- employee.department_name = department.department_name
- project.department_name = department.department_name
- employee.manager_name = project.manager_name

-----

### **TASK**

Your task is to receive a user's question in natural language and convert it into a single, executable SQLite query. Follow these steps meticulously:

1.  **Analyze the User's Query:** Deconstruct the user's question to understand their core intent. Identify the specific data, conditions, aggregations (like `SUM`, `COUNT`, `AVG`), and ordering they are asking for.
2.  **Map to the Schema:** Map the entities from the user's query to the appropriate tables and columns. Determine the necessary JOIN operations using employee.department_name = department.department_name, project.department_name = department.department_name, employee.manager_name = project.manager_name when needed.
3.  **Construct the SQLite Query:** Write a clean and efficient SELECT statement that is syntactically correct for SQLite. Ensure all table and column names are accurate.
4.  **Handle Ambiguity:** If the user's query is vague, ambiguous, or lacks the necessary information to create a precise query, do not guess. Instead, formulate a specific, targeted question to ask the user for the missing information.

-----

### **CONSTRAINTS**

  * **Read-Only Operations:** You must **ONLY** generate `SELECT` queries. Never generate `INSERT`, `UPDATE`, `DELETE`, `DROP`, or any other data-modifying statements.
  * **Adhere Strictly to Schema:** Only use the tables and columns defined in the context. Do not invent or assume the existence of any other tables or columns.
  * **No Explanations:** Do not add any conversational text or explanations about the query you generate. Your output must strictly follow the specified format.
  * **Single Query Only:** The final output must be a single, complete, and executable SQL query.
  * **Handle Impossibility:** If a request is impossible with the given schema, return an error explaining why.

-----

### **EXAMPLES**

**Example 1**
User Query: "List all employees in the HR department"
Output:
{
  "status": "success",
  "response": "SELECT e.first_name, e.last_name, e.email FROM employee e JOIN department d ON e.department_name = d.department_name WHERE d.department_name = 'HR';"
}


**Example 2**
User Query: "What is the average salary in each department?"
Output:
{
  "status": "success",
  "response": "SELECT d.department_name, AVG(e.salary) AS avg_salary FROM employee e JOIN department d ON e.department_name = d.department_name GROUP BY d.department_name;"
}

**Example 3**
User Query: "Show me recent employees"
Output:
{
  "status": "clarification_needed",
  "response": "Could you please specify what 'recent' means? For example, 'hired in the last 30 days' or 'hired this year'."
}

**Example 4**
User Query: "Which employee made the most sales?"
Output:
{
  "status": "error",
  "response": "I cannot answer this question as the database does not contain information about sales."
}

-----

### **OUTPUT FORMAT**

Your final response must be a single JSON object with two keys:

1.  `"status"`: A string with one of three possible values: `"success"`, `"clarification_needed"`, or `"error"`.
2.  `"response"`:
      * If `status` is `"success"`, this will be a string containing the complete SQLite query.
      * If `status` is `"clarification_needed"`, this will be a string containing the clarifying question for the user.
      * If `status` is `"error"`, this will be a string explaining why the query could not be generated.
"""

In [49]:
import json
def get_sql_query(genai_client, prompt, user_query):

  contents = f"""
  {prompt}

  Here's the user query in english you need to work on:
  {user_query}
  """
  response = genai_client.models.generate_content(model='gemini-2.5-flash', contents=contents)
  # print(response)

  # Access the usage_metadata attribute
  usage_metadata = response.usage_metadata

  # Print the different token counts
  print(f"Input Token Count: {usage_metadata.prompt_token_count}")
  print(f"Thoughts Token Count: {response.usage_metadata.thoughts_token_count}")
  print(f"Output Token Count: {usage_metadata.candidates_token_count}")
  print(f"Total Token Count: {usage_metadata.total_token_count}")

  output = json.loads(response.text.replace('```json', '').replace('```', ''))

  return output


In [50]:
import sqlite3
import pandas as pd

def execute_query(query, db_name='employees.db'):

    conn = None
    try:
        # Connect to the database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Execute the query
        print(f"\nExecuting query on '{db_name}':\n{query}")
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        # Get column names from the cursor description
        columns = [description[0] for description in cursor.description]

        # Format results as a dataframe for easier use
        results_as_dict = [dict(zip(columns, row)) for row in results]
        results_df = pd.DataFrame(results_as_dict)

        print("Query executed successfully.")
        return results_df

    except sqlite3.Error as e:
        print(f"Database error executing query: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None
    finally:
        if conn:
            conn.close()
            print("Database connection closed.")

In [51]:
def text2sql(genai_client, prompt, user_query):
  output = get_sql_query(genai_client, prompt, user_query)
  if output['status'] == 'success':
    results = execute_query(output['response'])
    return results
  return output

## Queries

In [52]:
text2sql(genai_client, prompt, "Show all the projects that start in the first half of the year")

Input Token Count: 1250
Thoughts Token Count: 185
Output Token Count: 44
Total Token Count: 1479

Executing query on 'employees.db':
SELECT * FROM project WHERE CAST(strftime('%m', start_date) AS INTEGER) <= 6;
Query executed successfully.
Database connection closed.


,project_id,start_date,end_date,manager_name,department_name
0,2,2022-04-27 00:00:00,2024-12-21 00:00:00,Brooks Beert,HR
1,4,2022-04-20 00:00:00,2025-05-18 00:00:00,Amandie Ambrosi,IT
2,5,2022-01-07 00:00:00,2025-04-30 00:00:00,Aleksandr Habbershon,Sales
3,9,2022-06-26 00:00:00,2025-01-22 00:00:00,Hermione Issard,Sales
4,10,2022-03-03 00:00:00,2025-08-19 00:00:00,Alphard Macveigh,Sales
...,...,...,...,...,...
498,988,2022-06-24 00:00:00,2024-10-28 00:00:00,Abraham Thorold,HR
499,989,2022-04-14 00:00:00,2024-10-06 00:00:00,Noella Townend,HR
500,993,2022-05-11 00:00:00,2025-03-22 00:00:00,Antone Chaffe,Finance
501,999,2022-05-21 00:00:00,2025-03-25 00:00:00,Kayley Louiset,Finance


In [54]:
text2sql(genai_client, prompt, "What is the highest salary in IT department?")

Input Token Count: 1246
Thoughts Token Count: 165
Output Token Count: 61
Total Token Count: 1472

Executing query on 'employees.db':
SELECT MAX(e.salary) AS highest_salary FROM employee e JOIN department d ON e.department_name = d.department_name WHERE d.department_name = 'IT';
Query executed successfully.
Database connection closed.


,highest_salary
0,9997886


In [57]:
text2sql(genai_client, prompt, "What is the most common hire_date for Sales department")

Input Token Count: 1248
Thoughts Token Count: 162
Output Token Count: 50
Total Token Count: 1460

Executing query on 'employees.db':
SELECT hire_date FROM employee WHERE department_name = 'Sales' GROUP BY hire_date ORDER BY COUNT(*) DESC LIMIT 1;
Query executed successfully.
Database connection closed.


,hire_date
0,2025-08-16 00:00:00


In [61]:
text2sql(genai_client, prompt, "What is the average salary for each department?")

Input Token Count: 1246
Thoughts Token Count: 169
Output Token Count: 65
Total Token Count: 1480

Executing query on 'employees.db':
SELECT d.department_name, AVG(e.salary) AS average_salary FROM employee e JOIN department d ON e.department_name = d.department_name GROUP BY d.department_name;
Query executed successfully.
Database connection closed.


,department_name,average_salary
0,Finance,5.226599e+06
1,HR,5.308247e+06
2,IT,5.320815e+06
3,Marketing,5.345889e+06
4,Sales,5.387382e+06


In [62]:
text2sql(genai_client, prompt, "Find employees whose salary is above their department’s average")

Input Token Count: 1248
Thoughts Token Count: 247
Output Token Count: 95
Total Token Count: 1590

Executing query on 'employees.db':
SELECT e.first_name, e.last_name, e.salary, e.department_name FROM employee e JOIN (SELECT department_name, AVG(salary) AS avg_salary FROM employee GROUP BY department_name) AS da ON e.department_name = da.department_name WHERE e.salary > da.avg_salary;
Query executed successfully.
Database connection closed.


,first_name,last_name,salary,department_name
0,Nikolos,Hindge,7721180,IT
1,Abby,Ogger,8986545,Finance
2,Reuven,Andreini,5990261,IT
3,Farly,Janosevic,7299227,Marketing
4,Corny,Faiers,6384710,Sales
...,...,...,...,...
497,Linoel,Chasle,9619630,Finance
498,Gerda,Dupoy,5637397,Sales
499,Orelie,Pimbley,8120708,Sales
500,Liva,Denington,6601670,Marketing


In [64]:
text2sql(genai_client, prompt, "List managers and how many direct reports they have (only managers with at least one report)")

Input Token Count: 1255
Thoughts Token Count: 1584
Output Token Count: 56
Total Token Count: 2895

Executing query on 'employees.db':
SELECT manager_name, COUNT(employee_id) AS number_of_direct_reports FROM employee WHERE manager_name IS NOT NULL GROUP BY manager_name;
Query executed successfully.
Database connection closed.


,manager_name,number_of_direct_reports
0,Abby Mainston,1
1,Abdul Connell,1
2,Abie Rikel,1
3,Abram Corben,1
4,Abram Proudley,1
...,...,...
710,Yul Squirrell,1
711,Yvon Guillerman,1
712,Zebedee Davenall,1
713,Zora McGrane,1


In [69]:
text2sql(genai_client, prompt, "Employees hired before December 1, 2024")

Input Token Count: 1249
Thoughts Token Count: 139
Output Token Count: 57
Total Token Count: 1445

Executing query on 'employees.db':
SELECT employee_id, first_name, last_name, hire_date FROM employee WHERE hire_date < '2024-12-01';
Query executed successfully.
Database connection closed.


,employee_id,first_name,last_name,hire_date
0,7,Reuven,Andreini,2024-10-15 00:00:00
1,9,Farly,Janosevic,2024-11-02 00:00:00
2,17,Edgar,Heighton,2024-11-11 00:00:00
3,22,Chaddie,Penreth,2024-10-28 00:00:00
4,24,Dorey,Ayerst,2024-11-26 00:00:00
...,...,...,...,...
176,978,Townie,Jekel,2024-10-04 00:00:00
177,982,Nissie,Byles,2024-09-25 00:00:00
178,988,Iona,Hartnup,2024-11-06 00:00:00
179,991,Tony,Dunbleton,2024-09-23 00:00:00


## Project Summary

This project demonstrates the use of prompt engineering with the Gemini API to build a Text-to-SQL solution for an employee database. The database consists of three tables: `employee`, `department`, and `project`. The project involves:

1.  **Data Retrieval:** Downloading sample data for employees, departments, and projects from Mockaroo.
2.  **Database Setup:** Creating an SQLite database and defining the schema for the three tables.
3.  **Data Loading:** Loading the downloaded data into the respective tables using pandas DataFrames.
4.  **API Key Setup:** Setting up the API key for the Gemini API.
5.  **Prompt Engineering:** Crafting a detailed prompt for the Gemini API to act as an expert SQLite Database Engineer for NL2SQL translation. The prompt includes the database schema, relationships, task instructions, constraints, and examples.
6.  **Querying:** Developing functions to interact with the Gemini API to get SQL queries from natural language questions and to execute those queries against the SQLite database.
7.  **Demonstration:** Executing several natural language queries to demonstrate the Text-to-SQL functionality and observing the generated SQL and the results.

The project successfully shows how a large language model can be fine-tuned through prompt engineering to perform specific tasks like converting natural language questions into database queries.